# 1. 누락된 데이터 다루기

## 1.1 테이블 형태 데이터에서 누락된 값 식별

In [21]:
import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))

df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [23]:
# 각 열의 누락된 값의 개수 확인하기

df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

## 1.2 누락된 값이 있는 샘플이나 특성 제외

In [24]:
# 누락된 값이 있는 '행' 삭제

df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [25]:
# 누락된 값이 있는 '열' 삭제

df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [27]:
# 모든 열이 NaN인 행 삭제

df.dropna(how='all')  # 모든 열이 NaN인 열이 없기 때문에 삭제되는 행 없음

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [28]:
# thresh 값보다 실수가 적은 행 삭제

df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [29]:
# 특정 열에 NaN이 있는 행만 삭제

df.dropna(subset=['C']) # NaN이 있는 2행이 삭제됨

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


## 1.3 누락된 값 대체

특성 열을 통째로 제거할 시, 유용한 데이터를 많이 잃기 쉽다. <br>
따라서 다른 훈련 샘플로부터 누락된 값을 추정할 수 있다.

1. 평균/중앙값으로 대체하기 ***(mean/median)***
2. 최빈값으로 대체하기 ***(most_frequent)***
3. 특정 값으로 채워넣기 ***(constant)***

In [31]:
# Inputer 클래스 이용
# Imputer(missing_values = 'NaN', strategy = 'mean','median', 'most_frequent', axis=0(열기준), axis(행기준))
# 0.22버전에 삭제될 예정

# 평균값 채워넣기
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [35]:
# 최빈값으로 채워넣기

from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6.,  3.,  8.],
       [10., 11., 12.,  4.]])

In [43]:
# SimpleImputer 클래스 이용
# SimpleImputer(missing_values = np.nan, strategy='mean', 'median', 'most_frequent', 'constant')
# missing_values의 기본값이 np.nan이기 때문에 import numpy를 해줘야 한다.
# SimpleImputer의 경우 axis가 없으며 열 기준으로 strategy가 적용된다.

import numpy as np
from sklearn.impute import SimpleImputer

# 평균값 채워넣기
simr = SimpleImputer(missing_values=np.nan, strategy='mean')
simr = simr.fit(df.values)
imputed_data = simr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [38]:
import numpy as np
from sklearn.impute import SimpleImputer

# 최빈값 채워넣기
simr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
simr = simr.fit(df.values)
imputed_data = simr.transform(df.values)
imputed_data

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6.,  3.,  8.],
       [10., 11., 12.,  4.]])

In [44]:
import numpy as np
from sklearn.impute import SimpleImputer

# 특정값 채워넣기
simr = SimpleImputer(missing_values=np.nan, strategy='constant')
simr = simr.fit(df.values)
imputed_data = simr.transform(df.values)
imputed_data

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6.,  0.,  8.],
       [10., 11., 12.,  0.]])

In [42]:
# FunctionTransformer 클래스 이용
# SimpleImputer의 입력 행과 열을 바꿔 strategy를 적용하여 계산 후 다시 변환된 열과 행을 바꿈

from sklearn.preprocessing import FunctionTransformer
ftr_simr = FunctionTransformer(lamda X: simr.fit_transform(X.T).T, validation=False)
imputed_data = ftr_simr.fit_transform(df.values)
imputed_data

SyntaxError: invalid syntax (<ipython-input-42-d7124451444f>, line 5)